In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys; print(sys.version)

3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]


In [3]:
import scipy.io as sio
from fastai.vision import *
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math
import time
from tqdm import tqdm

In [4]:
from model.res_net_use_this import ResNet, ResidualBlock, Bottleneck

In [5]:
batch_size = 20
num_epochs = 20
learning_rate = 0.001

In [6]:
!nvidia-smi

Thu Feb 21 12:20:02 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.91       Driver Version: 418.91       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108... WDDM  | 00000000:03:00.0  On |                  N/A |
| 23%   28C    P8    11W / 250W |    284MiB / 11264MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080   WDDM  | 00000000:04:00.0  On |                  N/A |
| 27%   31C    P8     7W / 180W |    664MiB /  8192MiB |     11%      Default |
+-------------------------------+----------------------+----------------------+
        

In [7]:
PATH=Path('data/')

In [8]:
sio.whosmat(PATH/f'aug_train_data.mat')

[('aug_train_data', (38633, 30), 'double'),
 ('aug_train_label', (38633, 9), 'double')]

In [9]:
mat = sio.loadmat(PATH/f'aug_train_data.mat'); mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Apr 10 13:06:16 2018',
 '__version__': '1.0',
 '__globals__': [],
 'aug_train_data': array([[ 30.986349,  66.696615,  89.191177, 107.440148, ...,  21.567924,  33.402839,  41.316166,  35.6314  ],
        [ 31.000555,  66.674004,  89.128681, 107.344902, ...,  21.528592,  33.400976,  41.291833,  35.60947 ],
        [ 31.016712,  66.656017,  89.072481, 107.257079, ...,  21.490221,  33.40014 ,  41.269528,  35.589458],
        [ 31.034781,  66.642589,  89.022504, 107.176601, ...,  21.452846,  33.400322,  41.249235,  35.571345],
        ...,
        [ 28.768423,  66.065946,  90.730008, 111.442611, ...,  28.044473,  12.206115,  22.673065,  22.486479],
        [ 32.304686,  72.489195,  98.974761, 120.000958, ...,  30.553175,  10.668108,  20.468008,  21.2884  ],
        [ 26.815675,  66.04399 ,  93.072998, 116.487214, ...,  29.530984,   7.738786,  21.665632,  22.855638],
        [ 30.255836,  66.656272,  90.221104, 109.6948

In [10]:
train_data = mat['aug_train_data']; train_data.shape

(38633, 30)

In [11]:
train_label = mat['aug_train_label']; train_label.shape

(38633, 9)

In [12]:
train_label[:, 0] = train_label[:, 0] - 1; train_label

array([[ 0.      ,  5.      , 89.7     , 65.1     , ...,  0.125   ,  0.882222,  0.717143,  0.722222],
       [ 0.      ,  5.      , 89.7     , 65.1     , ...,  0.125   ,  0.882222,  0.717143,  0.722222],
       [ 0.      ,  5.      , 89.7     , 65.1     , ...,  0.125   ,  0.882222,  0.717143,  0.722222],
       [ 0.      ,  5.      , 89.7     , 65.1     , ...,  0.125   ,  0.882222,  0.717143,  0.722222],
       ...,
       [29.      , 22.3     , 73.6     , 53.2     , ...,  0.5575  ,  0.524444,  0.377143,  0.133333],
       [29.      , 22.3     , 73.6     , 53.2     , ...,  0.5575  ,  0.524444,  0.377143,  0.133333],
       [29.      , 22.3     , 73.6     , 53.2     , ...,  0.5575  ,  0.524444,  0.377143,  0.133333],
       [29.      , 22.3     , 73.6     , 53.2     , ...,  0.5575  ,  0.524444,  0.377143,  0.133333]])

In [13]:
num_train_instances = len(train_data)
train_data = torch.from_numpy(train_data).type(torch.FloatTensor).view(num_train_instances, 30, 1, 1)
train_label = torch.from_numpy(train_label).type(torch.FloatTensor)
train_dataset = TensorDataset(train_data, train_label)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [14]:
sio.whosmat(PATH/f'test.mat')

[('test_data', (4444, 30), 'double'), ('test_label', (4444, 9), 'double')]

In [15]:
mat = sio.loadmat('data/test.mat'); mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Apr 09 16:09:11 2018',
 '__version__': '1.0',
 '__globals__': [],
 'test_data': array([[ 31.69325 ,  68.672322,  91.424062, 109.585392, ...,  13.151581,  26.322912,  36.42973 ,  32.152809],
        [ 31.677454,  68.663352,  91.424086, 109.596771, ...,  13.15013 ,  26.315097,  36.430637,  32.156283],
        [ 31.662161,  68.654645,  91.424127, 109.607841, ...,  13.148728,  26.307654,  36.43158 ,  32.159659],
        [ 31.647381,  68.646203,  91.42418 , 109.618587, ...,  13.147373,  26.300585,  36.432554,  32.162932],
        ...,
        [ 41.715149,  82.797009, 106.103027, 121.930894, ...,  30.928163,  11.535122,  17.998903,  19.699683],
        [ 41.651226,  82.777406, 106.143961, 122.04256 , ...,  30.887208,  11.405573,  18.036946,  19.766039],
        [ 41.583979,  82.753862, 106.181303, 122.152158, ...,  30.845885,  11.27573 ,  18.075906,  19.833085],
        [ 41.513455,  82.726378, 106.214998, 122.259574, .

In [16]:
test_data = mat['test_data']; test_data.shape

(4444, 30)

In [17]:
test_label = mat['test_label']; test_label.shape

(4444, 9)

In [18]:
test_label[:, 0] = test_label[:, 0] - 1; test_label

array([[ 0.      ,  5.      , 89.7     , 65.1     , ...,  0.125   ,  0.882222,  0.717143,  0.722222],
       [ 0.      ,  5.      , 89.7     , 65.1     , ...,  0.125   ,  0.882222,  0.717143,  0.722222],
       [ 0.      ,  5.      , 89.7     , 65.1     , ...,  0.125   ,  0.882222,  0.717143,  0.722222],
       [ 0.      ,  5.      , 89.7     , 65.1     , ...,  0.125   ,  0.882222,  0.717143,  0.722222],
       ...,
       [29.      , 22.3     , 73.6     , 53.2     , ...,  0.5575  ,  0.524444,  0.377143,  0.133333],
       [29.      , 22.3     , 73.6     , 53.2     , ...,  0.5575  ,  0.524444,  0.377143,  0.133333],
       [29.      , 22.3     , 73.6     , 53.2     , ...,  0.5575  ,  0.524444,  0.377143,  0.133333],
       [29.      , 22.3     , 73.6     , 53.2     , ...,  0.5575  ,  0.524444,  0.377143,  0.133333]])

In [19]:
num_test_instances = len(test_data)
test_data = torch.from_numpy(test_data).type(torch.FloatTensor).view(num_test_instances, 30, 1, 1)
test_label = torch.from_numpy(test_label).type(torch.FloatTensor)
test_dataset = TensorDataset(test_data, test_label)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [20]:
resnet = ResNet(ResidualBlock, [2, 2, 2, 2], 30)

In [21]:
resnet = resnet.cuda()

In [22]:
criterion1 = nn.CrossEntropyLoss().cuda()
criterion2 = nn.L1Loss().cuda()
optimizer = torch.optim.Adam(resnet.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 6, 9, 12, 15, 18], gamma=0.3)

In [23]:
for epoch in range(num_epochs):
    print('Epoch:', epoch)
    resnet.train()

    scheduler.step()
    # trained_num = 0
    for (samples, labels) in tqdm(train_data_loader):

        samplesV = Variable(samples.cuda())
        labels = labels.squeeze()
        labelsV = Variable(labels.cuda())

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        predict_label = resnet(samplesV)

        lossC = criterion1(predict_label[0], labelsV[:, 0].type(torch.LongTensor).cuda())

        lossR1 = criterion2(predict_label[1][:, 0], labelsV[:, 1])
        lossR2 = criterion2(predict_label[1][:, 1], labelsV[:, 2])
        lossR3 = criterion2(predict_label[1][:, 2], labelsV[:, 3])
        lossR4 = criterion2(predict_label[1][:, 3], labelsV[:, 4])

        loss = lossC + (0.0386*lossR1 + 0.0405*lossR2 + 0.0629*lossR3 + 0.0877*lossR4)/4
        loss.backward()
        optimizer.step()
        
    resnet.eval()
    correct_t = 0
    for (samples, labels) in tqdm(train_data_loader):
        with torch.no_grad():
            samplesV = Variable(samples.cuda())
            labelsV = Variable(labels.cuda())
            predict_label = resnet(samplesV)
            prediction = predict_label[0].data.max(1)[1]
            correct_t += prediction.eq(labelsV[:, 0].data.long()).sum()

    print("Training accuracy:", (100*float(correct_t)/num_train_instances))

    trainacc = str(100*float(correct_t)/num_train_instances)[0:6]

    correct_t = 0
    for (samples, labels) in tqdm(test_data_loader):
        with torch.no_grad():
            samplesV = Variable(samples.cuda())
            labelsV = Variable(labels.cuda())

        predict_label = resnet(samplesV)
        prediction = predict_label[0].data.max(1)[1]
        correct_t += prediction.eq(labelsV[:, 0].data.long()).sum()

    print("Test accuracy:", (100 * float(correct_t) / num_test_instances))

    testacc = str(100 * float(correct_t) / num_test_instances)[0:6]

    torch.save(resnet, 'weights/resnet18_Train' + trainacc + 'Test' + testacc + '.pkl')

Epoch: 0


100%|██████████████████████████████████████████████████████████████████████████████| 1932/1932 [00:57<00:00, 33.74it/s]


Training accuracy: 87.89118111459115


100%|████████████████████████████████████████████████████████████████████████████████| 223/223 [00:06<00:00, 32.91it/s]


Test accuracy: 68.15931593159316
Epoch: 1


...

Test accuracy: 85.35103510351036
Epoch: 19


100%|██████████████████████████████████████████████████████████████████████████████| 1932/1932 [00:58<00:00, 32.87it/s]


Training accuracy: 100.0


100%|████████████████████████████████████████████████████████████████████████████████| 223/223 [00:07<00:00, 31.67it/s]


Test accuracy: 85.37353735373537
